In [36]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.optimize import linprog, LinearConstraint, minimize
from scipy import constants

In [8]:
# Single or double insertion reaction
num_inserts = 1

# Total volume of plasmid and insert (in uL)
total_vol = 7

# Concentrations (in ng/uL)
plasmid_conc = 134.7
insert_conc = 59.6

# Molecular weights (in kDa)
plasmid_mw = 7746.122
insert_mw = 254.521

# kDa / ng ratio
kda_ng = 6.022e11

# Concentrations (in kDa/uL)
plasmid_conc_kda = plasmid_conc * kda_ng
insert_conc_kda = insert_conc * kda_ng

# Molar concentrations
plasmid_mconc = plasmid_conc_kda / plasmid_mw
insert_mconc = insert_conc_kda / insert_mw

# Ratio of insert volume to plasmid volume
insert_plasmid_ratio = (2 * plasmid_mconc) / insert_mconc

# Volumes of plasmid and insert to add (in uL) according to volume limit
plasmid_vol = total_vol / (insert_plasmid_ratio + 1)
insert_vol = total_vol - plasmid_vol



In [21]:
# Concentrations (in ng/uL)
plasmid_conc = 134.7
insert_conc = 59.6

# Molecular weights (in kDa)
plasmid_mw = 7746.122
insert_mw = 254.521

S = insert_mw / plasmid_mw

# Max mass of DNA in reaction (in ng)
max_mass = 200

# Max volume of DNA in reaction (in uL)
max_vol = 6



c = np.array([-plasmid_conc, -insert_conc])

A_ub = np.array([
    [plasmid_conc, 0],
    [0, insert_conc],
    [1, 1]
])

b_ub = np.array([200 / ((2 * S) + 1), (400 * S) / ((2 * S) + 1), max_vol])

bounds = (0.2, max_vol)

res = linprog(c, A_ub = A_ub, b_ub = b_ub, bounds = bounds)
res.x, res.fun

(array([1.39322426, 0.20692437]), -200.0)

In [35]:
# Concentrations (in ng/uL)
plasmid_conc = 11.9
insert_conc = 91.9

# Molecular weights (in kDa)
plasmid_mw = 7107.896
insert_mw = 860.023

R = 6.022e11

S = insert_mw / plasmid_mw

# Max mass of DNA in reaction (in ng)
max_mass = 200

# Max volume of DNA in reaction (in uL)
max_vol = 8



c = np.array([-plasmid_conc, -insert_conc])

A_ub = np.array([
    [plasmid_conc, 0],
    [0, insert_conc],
    [1, 1]
])

b_ub = np.array([200 / ((2 * S) + 1), (400 * S) / ((2 * S) + 1), max_vol])

bounds = (0.2, max_vol)

res = linprog(c, A_ub = A_ub, b_ub = b_ub, bounds = bounds)
print(res.x, res.fun)

print(moles(*res.x, plasmid_conc, insert_conc, plasmid_mw, insert_mw))
print(mass(*res.x, plasmid_conc, insert_conc, plasmid_mw, insert_mw))

[7.57597147 0.42402853] -129.1222821201363
(7638093643.175706, 27286087742.76043, 0.27992630219413744)
129.1222821201363


In [28]:
def moles(pv, iv, pc, ic, pmw, imw):
    plasmid_moles = (pc * R * pv) / pmw
    insert_moles = (ic * R * iv) / imw
    return plasmid_moles, insert_moles, plasmid_moles / insert_moles

def mass(pv, iv, pc, ic, pmw, imw):
    return (pc * pv) + (ic * iv)

In [47]:
# Concentrations (in ng/uL)
plasmid_conc = 11.9
insert_conc = 91.9

# Molecular weights (in kDa)
plasmid_mw = 7107.896
insert_mw = 860.023

R = constants.Avogadro / 1e12

S = insert_mw / plasmid_mw

# Max mass of DNA in reaction (in ng)
max_mass = 200

# Max volume of DNA in reaction (in uL)
max_vol = 8

# x = (pv, iv, pc, ic)
def total_dna(x):
    pv, iv, pc, ic = x
    return -((pv * pc) + (iv * ic))

cons = (
    {'type': 'ineq', 'fun': lambda x: max_vol - x[0] - x[1]},
    {'type': 'ineq', 'fun': lambda x: max_mass - (x[0] * x[2]) - (x[1] * x[3])},
    {'type': 'eq', 'fun': lambda x: (2 * S) - ((x[1] * x[3]) / (x[0] * x[2]))}
)

bnds = ((0.2, max_vol), (0.2, max_vol), (0, plasmid_conc), (0, insert_conc))

init_guess = (4, 4, 50, 50)

res = minimize(total_dna, init_guess, bounds = bnds, constraints = cons)
print(res.success, res.x, res.fun)

print(moles(*res.x, plasmid_mw, insert_mw))
print(mass(*res.x, plasmid_mw, insert_mw))

True [ 7.75693599  0.24306401 11.9        91.9       ] -114.6451206282415
(7820724851.504982, 15641449703.010435, 0.49999999999998496)
114.6451206282415
